<a href="https://colab.research.google.com/github/niksinfinity/ML_AI_Notebooks/blob/master/Keras__mnist_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.layers import Input,Dense,BatchNormalization
from keras import regularizers
from keras.models import Model
import numpy as np
import pandas as pd
import tensorflow
from keras.callbacks import TensorBoard


In [16]:
(train_x,train_y),(test_x,test_y) = mnist.load_data()

train_x,test_x = map(lambda x:np.array(x).reshape(x.shape[0],28,28,1),[train_x,test_x])
test_y,train_y = map(lambda x:np.array(x).reshape(x.shape[0],1),[test_y,train_y])
print(train_x.shape)
train_x = (train_x / 255)
test_x = (test_x / 255)
train_y = np_utils.to_categorical(train_y,10)
test_y = np_utils.to_categorical(test_y,10)

(60000, 28, 28, 1)


In [0]:
from sklearn.model_selection import train_test_split
train_x,train_dev_x,train_y,train_dev_y = train_test_split(train_x, train_y,train_size=0.9,test_size=0.1,random_state=0,shuffle=True)
test_x,dev_x,test_y,dev_y = train_test_split(test_x, test_y, random_state=0,shuffle=True)

In [18]:
print("Train_x--> {}\t\t Train_y {} \nTrain_dev_x -> {} \t Train_dev_y -> {} \nTest_x -> {} \t\t Test_y -> {} \nDev_x -> {} \t\t Dev_y -> {}".format(train_x.shape,train_y.shape,train_dev_x.shape,train_dev_y.shape,test_x.shape,test_y.shape,dev_x.shape,dev_y.shape))

Train_x--> (54000, 28, 28, 1)		 Train_y (54000, 10) 
Train_dev_x -> (6000, 28, 28, 1) 	 Train_dev_y -> (6000, 10) 
Test_x -> (7500, 28, 28, 1) 		 Test_y -> (7500, 10) 
Dev_x -> (2500, 28, 28, 1) 		 Dev_y -> (2500, 10)


In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [29]:
from keras.layers import Input,BatchNormalization,Dense,Dropout
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras import regularizers
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

K.clear_session()

inputs = Input(shape=(28,28,1,))

X = Conv2D(16, kernel_size=(3,3), strides=(1, 1), padding='valid', activation='relu',name='conv11')(inputs)
X = BatchNormalization(name='batch11')(X)
X = Conv2D(16, kernel_size=(3,3), strides=(1, 1), padding='valid', activation='relu',name='conv12')(X)
X = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='valid',name='maxpool11')(X)
X = BatchNormalization(name='batch12')(X)
X = Dropout(rate=0.25)(X)

X = Conv2D(32, kernel_size=(3,3), strides=(1, 1), padding='valid', activation='relu',name='conv21')(X)
X = BatchNormalization(name='batch21')(X)
X = Conv2D(32, kernel_size=(3,3), strides=(1, 1), padding='valid', activation='relu',name='conv22')(X)
X = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='valid',name='maxpool21')(X)
X = BatchNormalization(name='batch22')(X)
X = Dropout(rate=0.25)(X)

X = Flatten(name='flat1')(X)
X = Dense(512, activation='relu',name='dense1')(X)
X = Dropout(rate=0.25)(X)
X = Dense(1024, activation='relu',name='dense2')(X)
X = Dropout(rate=0.5)(X)

prediction = Dense(units=10,activation='softmax',name='output')(X)

model = Model(inputs=inputs,outputs=prediction)

model.compile(optimizer=Adam(lr=0.0001,decay=0.000002, amsgrad=True),loss='categorical_crossentropy',metrics = ['accuracy'])

# log_dir = '/tmp/log/'
# tfk = TensorBoard(log_dir=log_dir, histogram_freq=0,write_graph=True, write_images=True)
# import shutil,os
# if 'log' in os.listdir('/tmp/'):
#     shutil.rmtree('/tmp/log/')
    
# tbc = TensorBoard(log_dir='/tmp/log/', histogram_freq=10, batch_size=32, write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv11 (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
batch11 (BatchNormalization) (None, 26, 26, 16)        64        
_________________________________________________________________
conv12 (Conv2D)              (None, 24, 24, 16)        2320      
_________________________________________________________________
maxpool11 (MaxPooling2D)     (None, 23, 23, 16)        0         
_________________________________________________________________
batch12 (BatchNormalization) (None, 23, 23, 16)        64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 16)        0         
__________

In [30]:
model.fit_generator(generator=datagen.flow(train_x,train_y,batch_size=32),steps_per_epoch=500,epochs=300,shuffle=True,validation_data=(train_dev_x,train_dev_y),validation_steps=[10])

Epoch 1/300
500/500 [==============================] - 9s 18ms/step - loss: 0.7814 - acc: 0.7459 - val_loss: 0.1626 - val_acc: 0.9503
Epoch 2/300
500/500 [==============================] - 8s 15ms/step - loss: 0.3281 - acc: 0.9003 - val_loss: 0.1580 - val_acc: 0.9513
Epoch 3/300
500/500 [==============================] - 8s 15ms/step - loss: 0.2363 - acc: 0.9281 - val_loss: 0.1418 - val_acc: 0.9525
Epoch 4/300
500/500 [==============================] - 8s 15ms/step - loss: 0.1974 - acc: 0.9380 - val_loss: 0.2528 - val_acc: 0.9172
Epoch 5/300
500/500 [==============================] - 8s 15ms/step - loss: 0.1721 - acc: 0.9477 - val_loss: 0.1045 - val_acc: 0.9673
Epoch 6/300
500/500 [==============================] - 8s 15ms/step - loss: 0.1490 - acc: 0.9561 - val_loss: 0.0862 - val_acc: 0.9750
Epoch 7/300
500/500 [==============================] - 8s 15ms/step - loss: 0.1381 - acc: 0.9579 - val_loss: 0.0742 - val_acc: 0.9780
Epoch 8/300
500/500 [==============================] - 8s 15ms

In [31]:
# import json
# model_json = model.to_json()
# json.dump(model_json,open('cnn_mnist_model.json','w'),indent=4)
# model.save_weights("cnn_mnist_weights.h5")
# print("Saved model to disk")

print("Dev Data-->",model.evaluate(x=dev_x, y=dev_y, batch_size=32)[1])
#0.996
print("Test Data -->",model.evaluate(x=test_x, y=test_y, batch_size=32)[1])
# 0.9957333333333334
print("Train Data -->",model.evaluate(x=train_x, y=train_y, batch_size=32)[1])
#0.999
print("Train Dev Data -->",model.evaluate(x=train_dev_x, y=train_dev_y, batch_size=32)[1])
# 0.9965

2500/2500 [==============================] - 0s 91us/step
Dev Data--> 0.996
7500/7500 [==============================] - 1s 80us/step
Test Data --> 0.9957333333333334
54000/54000 [==============================] - 4s 68us/step
Train Data --> 0.999
6000/6000 [==============================] - 0s 66us/step
Train Dev Data --> 0.9965
